In [ ]:
#Includes main algorithm to calculate OTDGWD between datasets, the relevant comparison between distance and transferability error along with the associated graphs and figures
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics.cluster import completeness_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition)
import numpy as np
import time
from sklearn import cluster
from sklearn.neighbors import kneighbors_graph
from sklearn.utils import shuffle
from matplotlib import offsetbox
import math
import ot
import pickle
import pandas as pd
%store -r
from emnist import extract_training_samples
import cv2
import os

#loading transformed MNIST Datasets (see dataaugment.ipynb for details)
with open('dataaugment1.pickle', 'rb') as f:
    data_cropp, data_60, data_90, data_120, data_150, data_180= pickle.load(f,encoding='bytes')

#loading USPS Dataset
path_to_data = "./USPSdata/Numerals/"
def resize_and_scale(img, size):
    img = cv2.resize(img, size)
    return 1 - np.array(img, "float32")

img_list = os.listdir(path_to_data)
sz = (16,16)
validation_usps = []
validation_usps_label = []
for i in range(10):
    label_data = path_to_data + str(i) + '/'
    img_list = os.listdir(label_data)
    for name in img_list:
        if '.png' in name:
            img = cv2.imread(label_data+name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized_img = resize_and_scale(img, sz)
            validation_usps.append(resized_img.flatten())
            validation_usps_label.append(i)

validation_usps = np.array(validation_usps)

validation_usps_label= np.array(validation_usps_label)

#loading KMNIST
kmnist_data = np.load(r'C:\Users\IST\Downloads\kmnist-train-imgs.npz')

kmnist_data = kmnist_data.f.arr_0

labels_kmnist = np.load(r'C:\Users\IST\Downloads\kmnist-train-labels.npz')

labels_kmnist = labels_kmnist.f.arr_0

#loading MNIST
mnistt = fetch_openml('mnist_784')
#Loading FASHION-MNIST
fmnistt = fetch_openml('Fashion-MNIST') 
# loading EMNIST
images, labels = extract_training_samples('letters') #emnist

#get mean and covariance for each label distribution in the dataset
def label_params(df , target):
    feature_label_pairs = df.loc[(df['target' ]== target)]
    feature_label_pairs = feature_label_pairs.iloc[:,0:len(df.columns)-1]
    features = pd.DataFrame.to_numpy(feature_label_pairs)

    sum_mean = np.zeros(len(df.columns)-1)
    for i in range(len(features)-1):
        sum_mean = sum_mean + features[i]
    
    
    mu = sum_mean / len(features)

    sum_cov = np.zeros((len(df.columns)-1, len(df.columns)-1))
    for i in range(len(features)-1):
        sum_cov = sum_cov + np.outer(np.transpose(features[i]-mu), (features[i]-mu))

    
    cov = sum_cov / len(features)
    
    return [mu , cov]

#get distance between two labels
def label_dist(params_1 , params_2):
    w_2 = np.linalg.norm(params_1[0] - params_2[0]) + np.matrix.trace(params_1[1] + params_2[1] - 2*np.sqrt((np.sqrt(params_1[1])*(params_2[1])*np.sqrt(params_1[1]))))
    return w_2

#store mean and covariance for each label in a dictionary
def class_param(target, df):
    class_params = dict.fromkeys(np.unique(target))
    for key in class_params.keys():
       class_params[key] = label_params(df, key) 
    return class_params

#get pointwise pairwise distances between points in a dataset
def cost_mat(data, target , df ):
    print('here')
    class_params = class_param(target, df)
    cost_mat = np.zeros(( len(df), len(df) ))
    for i in range(0,len(df)):
#          print(i)
         target_1 = target[i]
         params_1 = class_params[target_1]
         for j in range(0,len(df)):
#                 print(j)
                target_2 = target[j]
                params_2 = class_params[target_2]
                w_2 = label_dist(params_1, params_2)
                cdist = np.linalg.norm(data[i] - data[j])
            
            
                if (cdist + w_2) >= 0:
                      pairwise_dist = math.sqrt(cdist + w_2)
                else:
                      pairwise_dist = 0.0
        
                cost_mat[i][j] = pairwise_dist
    
    return cost_mat




#get distance between two datasets, (C_A - Cost Matrix of Dataset A, C_B - Cost Matrix of Dataset B, epsilon - regularization parameter)
def results(C_A, C_B, epsilon):
    import matplotlib.pylab as pl
    from mpl_toolkits.mplot3d import Axes3D 
    
    C_A /= C_A.max()
    C_B /= C_B.max()

    p = ot.unif(len(C_A))
    q = ot.unif(len(C_B))
    
   
    #regularized entropic gromov wasserstein calculation
    gw, log = ot.gromov.entropic_gromov_wasserstein(
        C_A, C_B, p, q, 'square_loss', epsilon=epsilon, log=True, verbose=True)

    
    
    print('Entropic Gromov-Wasserstein distances: ' + str(log['gw_dist']))


    pl.figure(1, (10, 5))
    
    
    #plot of optimal coupling matrix for regularized entropic GW
    
    pl.subplot(1, 2, 2)
    pl.imshow(gw, cmap = 'nipy_spectral', interpolation='nearest')
    pl.title('Entropic Gromov Wasserstein ($\epsilon =$)', epsilon )
    plt.grid(color='red', linestyle='-.', linewidth=0.7)
    pl.show()
    
    distance = [gw, log]
    return distance

    
#creating and randomizing dataframes for all datasets       
   
mnist = pd.concat([pd.DataFrame(mnistt.data), pd.DataFrame({ 'target' : mnistt.target})], axis = 1)

mnist =shuffle(mnist, random_state = 2)

fmnist = pd.concat([pd.DataFrame(fmnistt.data), pd.DataFrame({ 'target' : fmnistt.target})], axis = 1)

fmnist =shuffle(fmnist, random_state = 2)

images = images.reshape((images.shape[0], 784))
emnist = pd.concat([pd.DataFrame(images), pd.DataFrame({ 'target' : labels})], axis = 1)
emnist = shuffle(emnist, random_state = 2)

kmnist_data = kmnist_data.reshape((kmnist_data.shape[0], 784))
kmnist = pd.concat([pd.DataFrame(kmnist_data), pd.DataFrame({ 'target' : labels_kmnist})], axis = 1)
kmnist = shuffle(kmnist, random_state = 2)   

usps = pd.concat([pd.DataFrame(validation_usps), pd.DataFrame({ 'target' : validation_usps_label})], axis = 1)

usps =shuffle(usps, random_state = 2)

validation_usps = pd.DataFrame(usps.iloc[:,0:784]).to_numpy()

validation_usps_label =usps.target

data_crop_1 = data_cropp.reshape((data_cropp.shape[0], 196))

mnist_crop = pd.concat([pd.DataFrame(data_crop_1), pd.DataFrame({ 'target' : mnistt.target})], axis = 1)

mnist_crop =shuffle(mnist_crop, random_state = 2)

dataset_crop = pd.DataFrame(mnist_crop.iloc[:,0:196]).to_numpy()

labels_crop = mnist_crop.target

data_60_1 = data_60.reshape((data_60.shape[0], 784))

mnist_60 = pd.concat([pd.DataFrame(data_60_1), pd.DataFrame({ 'target' : mnistt.target})], axis = 1)

mnist_60 =shuffle(mnist_60, random_state = 2)

dataset_60 = pd.DataFrame(mnist_60.iloc[:,0:784]).to_numpy()

labels_60 = mnist_60.target

data_120_1 = data_120.reshape((data_120.shape[0], 784))

mnist_120 = pd.concat([pd.DataFrame(data_120_1), pd.DataFrame({ 'target' : mnistt.target})], axis = 1)

mnist_120 =shuffle(mnist_120, random_state = 2)

dataset_120 = pd.DataFrame(mnist_120.iloc[:,0:784]).to_numpy()

labels_120 = mnist_120.target

data_180_1 = data_180.reshape((data_180.shape[0], 784))

mnist_180 = pd.concat([pd.DataFrame(data_180_1), pd.DataFrame({ 'target' : mnistt.target})], axis = 1)

mnist_180 =shuffle(mnist_180, random_state = 2)

dataset_180 = pd.DataFrame(mnist_180.iloc[:,0:784]).to_numpy()

labels_180 = mnist_180.target

def main():
     #calculating pairwise pointwise cost matrix for all datastes
     cost_mat_mnist = cost_mat(mnistt.data, mnistt.target , mnist)
     cost_mat_fmnist = cost_mat(fmnistt.data, fmnistt.target , fmnist)
     cost_mat_emnist = cost_mat(images, labels , emnist)
     cost_mat_kmnist = cost_mat(kmnist_data, labels_kmnist, kmnist)  
     cost_mat_usps2 = cost_mat(validation_usps, validation_usps_label, usps)
     cost_mat_crop = cost_mat(dataset_crop, labels_crop, mnist_crop)
     cost_mat_60 = cost_mat(dataset_60, labels_60, mnist_60)
     cost_mat_120 = cost_mat(dataset_120, labels_120, mnist_120)
     cost_mat_180 = cost_mat(dataset_180, labels_180, mnist_180)
        
   
  
     #Plotting Figure 3 (matrix containing dataset to dataset OTDGWD)
     sets_1 = ["MNIST", "EMNIST", "FMNIST", "KMNIST",
              "USPS"]
     sets_2 = ["MNIST", "EMNIST", "FMNIST", "KMNIST",
              "USPS"]

     entropicGWs = np.array([[0.0, float(str(results(cost_mat_emnist, cost_mat_mnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_mnist, cost_mat_fmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_mnist, cost_mat_kmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_mnist, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_emnist, cost_mat_mnist, 0.1)[1]['gw_dist'])[:5]), 0.0, float(str(results(cost_mat_emnist, cost_mat_fmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_emnist, cost_mat_kmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_emnist, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_fmnist, cost_mat_mnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_fmnist, cost_mat_emnist, 0.1)[1]['gw_dist'])[:5]), 0.0, float(str(results(cost_mat_fmnist, cost_mat_kmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_fmnist, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_kmnist, cost_mat_mnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_kmnist, cost_mat_emnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_kmnist, cost_mat_fmnist, 0.1)[1]['gw_dist'])[:5]), 0.0, float(str(results(cost_mat_kmnist, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_usps2, cost_mat_mnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_usps2, cost_mat_emnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_usps2, cost_mat_fmnist, 0.1)[1]['gw_dist'])[:5]), float(str(results(cost_mat_usps2, cost_mat_kmnist, 0.1)[1]['gw_dist'])[:5]), 0.0]])


     fig, ax = plt.subplots()
     im = ax.imshow(entropicGWs)

    # We want to show all ticks...
     ax.set_xticks(np.arange(len(sets_2)))
     ax.set_yticks(np.arange(len(sets_1)))
    # ... and label them with the respective list entries
     ax.set_xticklabels(sets_2)
     ax.set_yticklabels(sets_1)

    # Rotate the tick labels and set their alignment.
     plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
     for i in range(len(sets_2)):
         for j in range(len(sets_1)):
             text = ax.text(j, i, entropicGWs[i, j],
                        ha="center", va="center", color="b")

     ax.set_title("Dataset Distance")
     fig.tight_layout()
     plt.show()
    
     #Plotting Figure 4 for epsilon = 0.1 (column vector containing  OTDGWD between augmented MNIST datasets and USPS for robustness analysis)
    
     sets_1_augment = ["MNIST", "MNIST-60", "MNIST-120", "MNIST-180", "MNIST-CROP"]
     sets_2_augment = ["USPS"]

     entropicGWs_augment = np.array([[float(str(results(cost_mat_mnist, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_60, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_120, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_180, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])],
                    [float(str(results(cost_mat_crop, cost_mat_usps2, 0.1)[1]['gw_dist'])[:5])]])


     fig, ax = plt.subplots()
     im = ax.imshow(entropicGWs_augment)

# We want to show all ticks...
     ax.set_xticks(np.arange(len(sets_2_augment)))
     ax.set_yticks(np.arange(len(sets_1_augment)))
# ... and label them with the respective list entries
     ax.set_xticklabels(sets_2_augment)
     ax.set_yticklabels(sets_1_augment)

# Rotate the tick labels and set their alignment.
     plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
     for i in range(len(sets_1_augment)):
         for j in range(len(sets_2_augment)):
            text = ax.text(j, i, entropicGWs_augment[i, j],
                       ha="center", va="center", color="b")

     ax.set_title("Dataset Distance")
     fig.tight_layout()
     plt.show()
    
    #classification accuracy on target dataset using a neural network pre-trained on the source dataset (see Domain_Adaptation.ipynb for details and DA_results.txt for relevant results)
     DA_accuracy = [0.9077056050300598, 0.9072727560997009, 0.9040692448616028, 0.9322309494018555, 0.9223484992980957, 0.9295114874839783,0.9868000149726868, 0.9918000102043152, 0.9889000058174133, 0.9828511476516724, 0.9790014028549194, 0.9824747443199158, 0.8898484706878662, 0.872943639755249, 0.8988449573516846, 0.9278963804244995 ]
    #classification accuracy of the neural network with just training on the target dataset (see Domain_Adaptation.ipynb for details and DA_results.txt for relevant results)
     source_accuracy = [0.9210822582244873, 0.9335178732872009,0.9939000010490417, 0.9831818342208862,  0.9810990691184998 ]
    
    #calculating transferability error as defined in the Final Report
     DA_error = []
     for h in range(len(DA_accuracy)):
         DA_accuracy[h] = 1 - DA_accuracy[h]
     
     for j in range(len(source_accuracy)):
         source_accuracy[j] = 1 - source_accuracy[j]
     for i in range(len(DA_accuracy)):
        
         temp = 0
         if i < 3:
       
             temp = 100*((DA_accuracy[i] - source_accuracy[0]) / (source_accuracy[0]))
             DA_error.append(temp)
         elif i >= 3 and i < 6:
        
             temp = 100*((DA_accuracy[i] - source_accuracy[1]) / (source_accuracy[1]))
             DA_error.append(temp)
         elif i >= 6 and i < 9:
        
             temp = 100*((DA_accuracy[i] - source_accuracy[2]) / (source_accuracy[2]))
             DA_error.append(temp)
        
         elif i >= 9 and i < 12:
             temp = 100*((DA_accuracy[i] - source_accuracy[3]) / (source_accuracy[3]))
             DA_error.append(temp)
         elif i >= 12 and i < 16:
             temp = 100*((DA_accuracy[i] - source_accuracy[4]) / (source_accuracy[4]))
             DA_error.append(temp)
        
    
        
    #Plotting Figure 2 for epsilon - 0.1
     GWs_for_regression = np.array([results(cost_mat_mnist, cost_mat_fmnist, 0.1)[1]['gw_dist'], results(cost_mat_emnist, cost_mat_fmnist, 0.1)[1]['gw_dist'], results(cost_mat_kmnist, cost_mat_fmnist, 0.1)[1]['gw_dist'], results(cost_mat_mnist, cost_mat_emnist, 0.1)[1]['gw_dist'], results(cost_mat_fmnist, cost_mat_emnist, 0.1)[1]['gw_dist'], results(cost_mat_kmnist, cost_mat_emnist, 0.1)[1]['gw_dist'], results(cost_mat_fmnist, cost_mat_mnist, 0.1)[1]['gw_dist'], results(cost_mat_emnist, cost_mat_mnist, 0.1)[1]['gw_dist'], results(cost_mat_kmnist, cost_mat_mnist, 0.1)[1]['gw_dist'], results(cost_mat_mnist, cost_mat_kmnist, 0.1)[1]['gw_dist'], results(cost_mat_fmnist, cost_mat_kmnist, 0.1)[1]['gw_dist'], results(cost_mat_emnist, cost_mat_kmnist, 0.1)[1]['gw_dist'], results(cost_mat_mnist, cost_mat_usps2, 0.1)[1]['gw_dist'], results(cost_mat_fmnist, cost_mat_usps2, 0.1)[1]['gw_dist'], results(cost_mat_emnist, cost_mat_usps2, 0.1)[1]['gw_dist'], results(cost_mat_kmnist, cost_mat_usps2, 0.1)[1]['gw_dist']]) 
    
    
     df = pd.concat([pd.DataFrame(GWs_for_regression , columns = ['OTDGWD Distance Between Datasets']), pd.DataFrame(DA_error, columns = ['Relative Change in Classification Error'])], axis = 1)
     
     sns.set_style("white")
     gridobj = sns.lmplot(x='OTDGWD Distance Between Datasets', y='Relative Change in Classification Error', data=df, 
                     height=5, aspect=1.6, robust=True, palette='tab10', 
                     scatter_kws=dict(s=20, linewidths=.4, edgecolors='black'))
     #point labels
     n = ['M-F', 'E-F', 'K-F', 'M-E', 'F-E', 'K-E', 'F-M', 'E-M', 'K-M', 'M-K', 'F-K', 'E-K', 'M-U', 'F-U', 'E-U', 'K-U' ]

    #generating plots between distance and transferability error

     for i, txt in enumerate(n):
        plt.annotate(txt, (df['OTDGWD Distance Between Datasets'][i], df['Relative Change in Classification Error'][i]), (df['OTDGWD Distance Between Datasets'][i], df['Relative Change in Classification Error'][i]))
    
     gridobj.set(xlim=(0.09, 0.1), ylim=(0, 20))
     plt.show()

     #calculating and printing corresponding p-value
     X2 = sm.add_constant(GWs_for_regression)
     est = sm.OLS(DA_error, X2)
     print(est.fit().f_pvalue)



      
        
        
main()